# N-body simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm import tqdm

In [2]:
def get_grav_acc(x, m, G):
    """
    Calculate gravitational acceleration of all points.
        x: (n, 2) array of positions
        m: (n,) array of masses
        G: gravitational constant
    """
    n = x.shape[0]
    assert x.shape == (n, 2) and m.shape == (n,)

    epsilon = 5  # For buffering/smoothing effect

    # Calculate pairwise displacement vectors (x_i - x_j)
    dx = x[:,np.newaxis,:] - x[np.newaxis,:,:]  # Shape: (n, n, 2)
    d = np.linalg.norm(dx, axis=2)

    mapped_masses = np.broadcast_to(np.broadcast_to(m[:,np.newaxis], (n, n))[:,:,np.newaxis], (n, n, 2))
    F = G * dx * mapped_masses / (d**3 + epsilon)[:,:,np.newaxis]

    acc = np.sum(F, axis=0) / np.broadcast_to(m[:,np.newaxis], (n, 2))

    return acc

In [3]:
def update_system(x, v, m, dt, G):
    """
    Update points due to gravitational attraction.
        x:  (n, 2) array of positions
        v:  (n, 2) array of velocities
        m:  (n,) array of masses
        dt: time step
        G:  gravitational constant
    """
    n = x.shape[0]
    assert x.shape == (n, 2) and v.shape == (n, 2) and m.shape == (n,)
    
    # Update positions and velocities using Verlet integration
    # https://en.wikipedia.org/wiki/Verlet_integration#Velocity_Verlet
    a = get_grav_acc(x, m, G)

    x_new = x + v * dt + 0.5 * a * dt**2
    a_new = get_grav_acc(x_new, m, G)
    v_new = v + 0.5 * (a + a_new) * dt

    return x_new, v_new

In [4]:
import imageio
import cv2

n = 1000
width = 512
height = 512
G = 1

theta = np.random.random((n,)) * (2 * np.pi)
r = (np.random.random((n,)) * 0.8 + 0.1) * (width/2)
x = np.array([np.cos(theta) * r + (width/2), np.sin(theta) * r + (height/2)]).T
v = np.zeros((n, 2))
m = np.ones((n,))

# Chonky boi
x[0] = [width/2, height/2]
m[0] = 100000
for i in range(1, n):
    dx = x[i] - x[0]
    dx = np.array([dx[1], -dx[0]])
    r = np.linalg.norm(dx)
    dx *= 300 * (1/r) * r**(-1/2)
    v[i] = dx
v[0] = [0, 0]

In [5]:
frames = []
fig, ax = plt.subplots(figsize=(width / 100, height / 100), dpi=100)
fig.patch.set_facecolor('black')

for i in tqdm(range(30)):
    x, v = update_system(x, v, m, 0.1, G)
    ax.clear()
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.scatter(x[:,0], x[:,1], color="white", s=np.sqrt(m))
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.buffer_rgba(), dtype="uint8")
    image = image.reshape(fig.canvas.get_width_height() + (4,))
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
    # Make the image writable by creating a copy
    image = image.copy()
    frames.append(image)

plt.close()

# Save frames as an animated GIF with looping
imageio.mimsave("nbody_test.gif", frames, fps=30, loop=0)

100%|██████████| 30/30 [00:07<00:00,  3.84it/s]
